## 1. Load Real Instacart Data

In [6]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Feature importance libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")

Libraries imported successfully!


In [ ]:
# AUTO SETUP FOR GOOGLE COLAB
print("🚀 Auto-detecting environment and setting up...")

import os
from pathlib import Path

# Check if running in Google Colab
try:
    from google.colab import drive
    IN_COLAB = True
    print("✅ Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("⚠️ Not in Colab - using local environment")

if IN_COLAB:
    # Auto mount Google Drive
    if not os.path.exists('/content/drive'):
        print("🔄 Mounting Google Drive...")
        try:
            drive.mount('/content/drive')
            print("✅ Drive mounted successfully!")
        except Exception as e:
            print(f"❌ Mount failed: {e}")
            print("💡 Please run: drive.mount('/content/drive') manually")
    else:
        print("✅ Drive already mounted")

# Install missing packages for Colab
if IN_COLAB:
    print("📦 Installing packages for Colab...")
    import subprocess
    import sys
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "plotly", "xgboost"])
        print("✅ Packages installed successfully!")
    except:
        print("⚠️ Some packages may already be installed")
else:
    print("⚠️ Skipping package installation for local environment")

# Smart path detection
print("\n📂 Detecting data paths...")
possible_paths = [
    ("/content/drive/MyDrive/instacart_data/raw", "/content/drive/MyDrive/instacart_data/processed"),  # Colab Drive
    ("/content/drive/My Drive/instacart_data/raw", "/content/drive/My Drive/instacart_data/processed"),  # Space in name
    ("/content/data/raw", "/content/data/processed"),  # Direct upload to Colab
    ("../data/raw", "../data/processed"),  # Local
]

data_raw = None
data_processed = None

for raw_path, proc_path in possible_paths:
    if Path(raw_path).exists():
        data_raw = Path(raw_path)
        data_processed = Path(proc_path)
        print(f"✅ Found data at: {data_raw}")
        break
    else:
        print(f"❌ Not found: {raw_path}")

if data_raw is None:
    # Default to Colab path if in Colab, local otherwise
    if IN_COLAB:
        data_raw = Path("/content/drive/MyDrive/instacart_data/raw")
        data_processed = Path("/content/drive/MyDrive/instacart_data/processed")
    else:
        data_raw = Path("../data/raw")
        data_processed = Path("../data/processed")
    print(f"⚠️ Using default path: {data_raw}")

# Create processed directory
try:
    data_processed.mkdir(parents=True, exist_ok=True)
    print(f"✅ Processed directory ready: {data_processed}")
except Exception as e:
    print(f"⚠️ Could not create processed directory: {e}")

# Check data availability
print(f"\n📂 Checking data availability...")
required_files = ["orders.csv", "order_products__prior.csv", "order_products__train.csv", 
                 "products.csv", "departments.csv", "aisles.csv"]

if data_raw.exists():
    print("✅ Raw data directory found!")
    found_files = list(data_raw.glob("*.csv"))
    print(f"📁 Found {len(found_files)} CSV files:")
    
    missing_files = []
    for req_file in required_files:
        file_path = data_raw / req_file
        if file_path.exists():
            file_size = file_path.stat().st_size / (1024*1024)  # MB
            print(f"   ✅ {req_file} ({file_size:.1f} MB)")
        else:
            missing_files.append(req_file)
            print(f"   ❌ {req_file} - MISSING")
    
    if missing_files:
        print(f"\n⚠️ Missing {len(missing_files)} required files:")
        for f in missing_files:
            print(f"   - {f}")
        print("\n🔧 Solutions:")
        if IN_COLAB:
            print("   1. Ensure shared folder is copied to 'My Drive'")
            print("   2. Check folder structure: MyDrive/instacart_data/raw/")
            print("   3. Verify all files were uploaded completely")
        else:
            print("   1. Check ../data/raw/ directory exists")
            print("   2. Verify file permissions")
    else:
        print(f"\n🎉 All required files found and ready!")
else:
    print("❌ Raw data directory not found!")
    if IN_COLAB:
        print("🔧 For Google Colab:")
        print("   1. Copy the shared folder to your 'My Drive'")
        print("   2. Rename folder to exactly 'instacart_data'")
        print("   3. Ensure structure: MyDrive/instacart_data/raw/*.csv")
        print("   4. Try manual mount cell below if needed")
    else:
        print("🔧 For local environment:")
        print("   1. Ensure data is in ../data/raw/ directory")
        print("   2. Check file permissions")

print(f"\n🎯 Setup complete!")
print(f"📍 Current paths:")
print(f"   Raw: {data_raw}")
print(f"   Processed: {data_processed}")

🚀 Setting up Google Colab environment...
⚠️ Drive mount issue: mount failed
💡 You may need to authorize access manually
✅ Required packages already available!
⚠️ Drive path not found, trying local paths...
✅ Processed directory created: ../data/processed

📂 Checking data availability...
Raw data path: ../data/raw
Processed data path: ../data/processed
❌ Raw data directory not found!
🔧 Please check data location or upload to correct path
📋 Required files: orders.csv, order_products__prior.csv, order_products__train.csv, products.csv, departments.csv, aisles.csv

🎯 Setup complete!


In [13]:
# Load REAL Instacart data for feature engineering
print("LOADING REAL INSTACART DATA FOR FEATURE EXPLORATION")
print("="*60)

# Use paths from setup cell (already defined globally)
print(f"📂 Using data paths:")
print(f"   Raw: {data_raw}")
print(f"   Processed: {data_processed}")

# Ensure processed directory exists
data_processed.mkdir(parents=True, exist_ok=True)

print("\nLoading real Instacart datasets...")

# Load raw data files (the complete real dataset)
try:
    # Core datasets with progress indicators
    print("📊 Loading orders.csv...")
    orders = pd.read_csv(data_raw / "orders.csv")
    print(f"   ✓ Loaded {orders.shape[0]:,} orders")
    
    print("📊 Loading order_products__prior.csv...")
    order_products_prior = pd.read_csv(data_raw / "order_products__prior.csv")
    print(f"   ✓ Loaded {order_products_prior.shape[0]:,} prior order-products")
    
    print("📊 Loading order_products__train.csv...")
    order_products_train = pd.read_csv(data_raw / "order_products__train.csv")
    print(f"   ✓ Loaded {order_products_train.shape[0]:,} train order-products")
    
    print("📊 Loading products.csv...")
    products = pd.read_csv(data_raw / "products.csv")
    print(f"   ✓ Loaded {products.shape[0]:,} products")
    
    print("📊 Loading departments.csv...")
    departments = pd.read_csv(data_raw / "departments.csv")
    print(f"   ✓ Loaded {departments.shape[0]:,} departments")
    
    print("📊 Loading aisles.csv...")
    aisles = pd.read_csv(data_raw / "aisles.csv")
    print(f"   ✓ Loaded {aisles.shape[0]:,} aisles")
    
    print("\n✅ Successfully loaded all raw data files!")
    
    # Display dataset sizes and insights
    print(f"\n📈 REAL DATASET SUMMARY:")
    print(f"  📦 Orders: {orders.shape[0]:,}")
    print(f"  👥 Users: {orders['user_id'].nunique():,}")
    print(f"  🛒 Products: {products.shape[0]:,}")
    print(f"  📋 Prior order-product records: {order_products_prior.shape[0]:,}")
    print(f"  🎯 Train order-product records: {order_products_train.shape[0]:,}")
    print(f"  🏪 Departments: {departments.shape[0]:,}")
    print(f"  🛍️ Aisles: {aisles.shape[0]:,}")
    
    # Real data patterns analysis
    print(f"\n📊 DATA PATTERNS ANALYSIS:")
    
    # Calculate basket sizes from prior orders
    prior_basket_sizes = order_products_prior.groupby('order_id').size()
    avg_basket_size = prior_basket_sizes.mean()
    
    # Calculate reorder rate
    reorder_rate = order_products_prior['reordered'].mean()
    
    # Calculate orders per user (only prior orders)
    prior_orders = orders[orders['eval_set'] == 'prior']
    orders_per_user = prior_orders.groupby('user_id').size()
    avg_orders_per_user = orders_per_user.mean()
    
    print(f"  🛒 Average basket size: {avg_basket_size:.2f} items")
    print(f"  🔄 Reorder rate: {reorder_rate:.1%}")
    print(f"  📊 Average orders per user: {avg_orders_per_user:.2f}")
    print(f"  📈 Max orders per user: {orders_per_user.max()}")
    print(f"  📉 Min orders per user: {orders_per_user.min()}")
    
    # Time patterns
    print(f"\n⏰ TIME PATTERNS:")
    peak_hours = orders['order_hour_of_day'].mode().values
    popular_days = orders['order_dow'].mode().values
    print(f"  🕙 Peak shopping hours: {peak_hours}")
    print(f"  📅 Most popular shopping days: {popular_days}")
    
    # Product insights
    print(f"\n🏆 TOP PRODUCTS:")
    top_products = order_products_prior['product_id'].value_counts().head(5)
    for product_id, count in top_products.items():
        product_name = products[products['product_id'] == product_id]['product_name'].iloc[0]
        print(f"    {product_name}: {count:,} orders")
    
    # Memory usage info
    total_memory = (orders.memory_usage(deep=True).sum() + 
                   order_products_prior.memory_usage(deep=True).sum() + 
                   products.memory_usage(deep=True).sum()) / 1024**2
    print(f"\n💾 Memory usage: {total_memory:.1f} MB")
    
    print(f"\n🎉 Real Instacart data loaded successfully!")
    print("🚀 Ready for feature engineering with authentic data patterns!")
    
except FileNotFoundError as e:
    print(f"❌ ERROR: {e}")
    print(f"\n🔧 TROUBLESHOOTING:")
    if IN_COLAB:
        print("For Google Colab:")
        print("1. Ensure the shared folder is added to your Drive")
        print("2. Check the folder name is exactly 'instacart_data'")
        print("3. Verify the structure: MyDrive/instacart_data/raw/*.csv")
        print("4. Try running the manual mount cell")
        print("5. Check file sharing permissions")
    else:
        print("For local environment:")
        print("1. Verify ../data/raw/ directory exists")
        print("2. Check all 6 CSV files are present")
        print("3. Verify file permissions")
    
    print(f"\n📋 Required files in {data_raw}:")
    required = ["orders.csv", "order_products__prior.csv", "order_products__train.csv", 
               "products.csv", "departments.csv", "aisles.csv"]
    for f in required:
        print(f"   - {f}")
        
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    print("💡 Try restarting the runtime and running cells again")
    raise e

LOADING REAL INSTACART DATA FOR FEATURE EXPLORATION
Loading real Instacart datasets...
Loading orders.csv...
❌ Error loading data: [Errno 2] No such file or directory: '/content/drive/MyDrive/instacart_data/raw/orders.csv'
Please check that all data files exist in the raw data directory


## 2. Xây dựng User-Level Features

**User-level features** mô tả hành vi mua sắm tổng thể của từng user:
- Số đơn đã mua
- Tần suất mua hàng 
- Khoảng cách giữa các đơn hàng
- Số sản phẩm đã mua
- Tỉ lệ reorder chung của user

In [4]:
print("BUILDING USER-LEVEL FEATURES")
print("="*40)

# Get prior orders for feature engineering
prior_orders = orders[orders['eval_set'] == 'prior'].copy()

# 1. Số đơn đã mua (Number of orders per user)
user_orders_count = prior_orders.groupby('user_id').size().reset_index(name='user_total_orders')

# 2. Tần suất mua hàng (Average days between orders)
user_avg_days_between_orders = prior_orders.groupby('user_id')['days_since_prior_order'].mean().reset_index()
user_avg_days_between_orders.columns = ['user_id', 'user_avg_days_between_orders']
user_avg_days_between_orders['user_avg_days_between_orders'] = user_avg_days_between_orders['user_avg_days_between_orders'].fillna(0)

# 3. Khoảng cách tối thiểu và tối đa giữa các đơn hàng
user_days_stats = prior_orders.groupby('user_id')['days_since_prior_order'].agg(['min', 'max', 'std']).reset_index()
user_days_stats.columns = ['user_id', 'user_min_days_between_orders', 'user_max_days_between_orders', 'user_std_days_between_orders']
user_days_stats = user_days_stats.fillna(0)

# 4. Thời gian mua sắm ưa thích
user_hour_mode = prior_orders.groupby('user_id')['order_hour_of_day'].apply(lambda x: x.mode().iloc[0] if not x.empty else 10).reset_index()
user_hour_mode.columns = ['user_id', 'user_preferred_hour']

user_dow_mode = prior_orders.groupby('user_id')['order_dow'].apply(lambda x: x.mode().iloc[0] if not x.empty else 0).reset_index()
user_dow_mode.columns = ['user_id', 'user_preferred_dow']

# 5. Số sản phẩm đã mua (từ order_products_prior)
# Total products per user
user_products_count = order_products_prior.groupby('order_id')['product_id'].count().reset_index()
user_products_count = user_products_count.merge(prior_orders[['order_id', 'user_id']], on='order_id')
user_total_products = user_products_count.groupby('user_id')['product_id'].sum().reset_index()
user_total_products.columns = ['user_id', 'user_total_products']

# Average basket size per user
user_avg_basket_size = user_products_count.groupby('user_id')['product_id'].mean().reset_index()
user_avg_basket_size.columns = ['user_id', 'user_avg_basket_size']

# 6. Unique products per user
user_unique_products = order_products_prior.merge(prior_orders[['order_id', 'user_id']], on='order_id')
user_unique_products_count = user_unique_products.groupby('user_id')['product_id'].nunique().reset_index()
user_unique_products_count.columns = ['user_id', 'user_unique_products']

# 7. Tỉ lệ reorder chung của user
user_reorder_rate = user_unique_products.groupby('user_id')['reordered'].mean().reset_index()
user_reorder_rate.columns = ['user_id', 'user_reorder_rate']

# Combine all user features
user_features = user_orders_count
user_features = user_features.merge(user_avg_days_between_orders, on='user_id', how='left')
user_features = user_features.merge(user_days_stats, on='user_id', how='left')
user_features = user_features.merge(user_hour_mode, on='user_id', how='left')
user_features = user_features.merge(user_dow_mode, on='user_id', how='left')
user_features = user_features.merge(user_total_products, on='user_id', how='left')
user_features = user_features.merge(user_avg_basket_size, on='user_id', how='left')
user_features = user_features.merge(user_unique_products_count, on='user_id', how='left')
user_features = user_features.merge(user_reorder_rate, on='user_id', how='left')

# Fill missing values
user_features = user_features.fillna(0)

print(f"✓ Created user-level features for {user_features.shape[0]:,} users")
print(f"  Features shape: {user_features.shape}")
print(f"  Feature columns: {list(user_features.columns)}")

# Display sample statistics
print(f"\nUser Features Summary:")
print(user_features.describe())
user_features.to_csv(data_processed / "user_features.csv", index=False)

BUILDING USER-LEVEL FEATURES


NameError: name 'orders' is not defined

## 3. Xây dựng Item-Level Features

**Item-level features** mô tả đặc điểm của từng sản phẩm:
- Độ phổ biến của sản phẩm
- Tỉ lệ reorder của sản phẩm
- Mức độ được thêm vào giỏ (add_to_cart_order)
- Thời điểm sản phẩm thường được mua

In [ ]:
print("BUILDING ITEM-LEVEL FEATURES")
print("="*40)

# 1. Độ phổ biến của sản phẩm (Product popularity)
product_popularity = order_products_prior['product_id'].value_counts().reset_index()
product_popularity.columns = ['product_id', 'product_total_orders']

# 2. Tỉ lệ reorder của sản phẩm
product_reorder_rate = order_products_prior.groupby('product_id')['reordered'].agg(['mean', 'sum', 'count']).reset_index()
product_reorder_rate.columns = ['product_id', 'product_reorder_rate', 'product_reorder_count', 'product_total_purchases']

# 3. Vị trí trung bình trong giỏ hàng (add_to_cart_order)
product_cart_position = order_products_prior.groupby('product_id')['add_to_cart_order'].mean().reset_index()
product_cart_position.columns = ['product_id', 'product_avg_cart_position']

# 4. Thời điểm sản phẩm thường được mua
# Merge với orders để có thông tin thời gian
order_products_with_time = order_products_prior.merge(
    prior_orders[['order_id', 'order_hour_of_day', 'order_dow']], 
    on='order_id'
)

# Giờ ưa thích mua sản phẩm
product_preferred_hour = order_products_with_time.groupby('product_id')['order_hour_of_day'].apply(
    lambda x: x.mode().iloc[0] if not x.empty else 10
).reset_index()
product_preferred_hour.columns = ['product_id', 'product_preferred_hour']

# Ngày trong tuần ưa thích
product_preferred_dow = order_products_with_time.groupby('product_id')['order_dow'].apply(
    lambda x: x.mode().iloc[0] if not x.empty else 0
).reset_index()
product_preferred_dow.columns = ['product_id', 'product_preferred_dow']

# 5. Số lượng users đã mua sản phẩm này
product_users_count = order_products_with_time.groupby('product_id')['order_id'].apply(
    lambda x: len(prior_orders[prior_orders['order_id'].isin(x)]['user_id'].unique())
).reset_index()
product_users_count.columns = ['product_id', 'product_unique_users']

# 6. Department và Aisle features
products_with_categories = products.merge(departments, on='department_id', how='left')
products_with_categories = products_with_categories.merge(aisles, on='aisle_id', how='left')

# Combine all product features
product_features = products_with_categories[['product_id', 'aisle_id', 'department_id']]
product_features = product_features.merge(product_popularity, on='product_id', how='left')
product_features = product_features.merge(product_reorder_rate, on='product_id', how='left')
product_features = product_features.merge(product_cart_position, on='product_id', how='left')
product_features = product_features.merge(product_preferred_hour, on='product_id', how='left')
product_features = product_features.merge(product_preferred_dow, on='product_id', how='left')
product_features = product_features.merge(product_users_count, on='product_id', how='left')

# Fill missing values
product_features = product_features.fillna(0)

print(f"✓ Created item-level features for {product_features.shape[0]:,} products")
print(f"  Features shape: {product_features.shape}")
print(f"  Feature columns: {list(product_features.columns)}")

# Display sample statistics
print(f"\nProduct Features Summary:")
print(product_features.describe())

# Save product features
print(f"\n💾 Saving product features to processed data...")
product_features.to_csv(data_processed / "product_features.csv", index=False)
product_features.to_parquet(data_processed / "product_features.parquet", index=False)
print(f"✓ Product features saved successfully!")

BUILDING ITEM-LEVEL FEATURES


KeyboardInterrupt: 

## 4. Xây dựng User-Product Interaction Features

**User-Product Interaction features** mô tả mối quan hệ giữa user và sản phẩm:
- Lịch sử mua hàng user-product
- Thời gian từ lần mua cuối
- Tần suất mua sản phẩm cụ thể
- Vị trí trung bình sản phẩm trong giỏ hàng của user

In [ ]:
print("BUILDING USER-PRODUCT INTERACTION FEATURES")
print("="*50)

# Lấy dữ liệu train để tạo target
train_orders = orders[orders['eval_set'] == 'train'].copy()
train_data = order_products_train.merge(train_orders[['order_id', 'user_id']], on='order_id')

print(f"Train data shape: {train_data.shape}")
print(f"Train users: {train_data['user_id'].nunique():,}")

# Tạo user-product interaction features từ prior orders
user_product_prior = order_products_prior.merge(
    prior_orders[['order_id', 'user_id', 'order_number']], 
    on='order_id'
)

print("Creating user-product interaction features...")

# 1. Lịch sử mua hàng user-product (đã từng mua hay chưa)
user_product_history = user_product_prior.groupby(['user_id', 'product_id']).agg({
    'order_id': 'count',  # số lần mua
    'reordered': 'sum',   # số lần reorder
    'add_to_cart_order': 'mean',  # vị trí trung bình trong giỏ
    'order_number': 'max'  # lần mua cuối cùng
}).reset_index()

user_product_history.columns = ['user_id', 'product_id', 'up_orders_count', 
                                'up_reorder_count', 'up_avg_cart_position', 
                                'up_last_order_number']

# 2. Thời gian từ lần mua cuối (days since last purchase)
user_max_order_number = prior_orders.groupby('user_id')['order_number'].max().reset_index()
user_max_order_number.columns = ['user_id', 'user_max_order_number']

user_product_history = user_product_history.merge(user_max_order_number, on='user_id')
user_product_history['up_days_since_last_purchase'] = user_product_history['user_max_order_number'] - user_product_history['up_last_order_number']

# 3. Tỉ lệ user mua sản phẩm này (order frequency ratio)
user_product_history = user_product_history.merge(
    user_features[['user_id', 'user_total_orders']], on='user_id'
)
user_product_history['up_order_rate'] = user_product_history['up_orders_count'] / user_product_history['user_total_orders']

# 4. Tỉ lệ reorder của user cho sản phẩm này
user_product_history['up_reorder_rate'] = user_product_history['up_reorder_count'] / user_product_history['up_orders_count']
user_product_history['up_reorder_rate'] = user_product_history['up_reorder_rate'].fillna(0)

print(f"✓ Created user-product interaction features")
print(f"  Shape: {user_product_history.shape}")
print(f"  Unique user-product pairs: {user_product_history.shape[0]:,}")

# Display sample
print(f"\nUser-Product Features Sample:")
print(user_product_history.head())

# Save user-product features
print(f"\n💾 Saving user-product interaction features...")
user_product_history.to_csv(data_processed / "user_product_features.csv", index=False)
user_product_history.to_parquet(data_processed / "user_product_features.parquet", index=False)
print(f"✓ User-product features saved successfully!")

## 5. Tạo Training Dataset cho Machine Learning

Kết hợp tất cả features để tạo dataset cuối cùng cho training:
- Merge user features, product features, và user-product interaction features
- Tạo target variable (reordered = 1/0)
- Tạo negative samples (sản phẩm user chưa từng mua)

In [ ]:
print("CREATING FINAL TRAINING DATASET")
print("="*40)

# 1. Tạo positive samples từ train data (những sản phẩm được reorder)
positive_samples = train_data[['user_id', 'product_id', 'reordered']].copy()
print(f"Positive samples: {positive_samples.shape[0]:,}")
print(f"Reordered products: {positive_samples['reordered'].sum():,}")
print(f"Reorder rate in train: {positive_samples['reordered'].mean():.3f}")

# 2. Merge với user features
final_dataset = positive_samples.merge(user_features, on='user_id', how='left')
print(f"After merging user features: {final_dataset.shape}")

# 3. Merge với product features  
final_dataset = final_dataset.merge(product_features, on='product_id', how='left')
print(f"After merging product features: {final_dataset.shape}")

# 4. Merge với user-product interaction features
final_dataset = final_dataset.merge(user_product_history, on=['user_id', 'product_id'], how='left')
print(f"After merging user-product features: {final_dataset.shape}")

# 5. Fill missing values for products that users never bought before
interaction_features = ['up_orders_count', 'up_reorder_count', 'up_avg_cart_position', 
                       'up_last_order_number', 'up_days_since_last_purchase', 
                       'up_order_rate', 'up_reorder_rate']

for feature in interaction_features:
    if feature in final_dataset.columns:
        if 'rate' in feature or 'position' in feature:
            final_dataset[feature] = final_dataset[feature].fillna(0)
        elif 'days_since' in feature:
            final_dataset[feature] = final_dataset[feature].fillna(999)  # very large number for never bought
        else:
            final_dataset[feature] = final_dataset[feature].fillna(0)

# Fill other missing values
final_dataset = final_dataset.fillna(0)

print(f"\n✓ Final training dataset created!")
print(f"  Shape: {final_dataset.shape}")
print(f"  Features: {final_dataset.shape[1] - 1}")  # -1 for target column
print(f"  Users: {final_dataset['user_id'].nunique():,}")
print(f"  Products: {final_dataset['product_id'].nunique():,}")
print(f"  Reorder rate: {final_dataset['reordered'].mean():.3f}")

# Display feature list
feature_columns = [col for col in final_dataset.columns if col not in ['user_id', 'product_id', 'reordered']]
print(f"\nFeature columns ({len(feature_columns)}):")
for i, feature in enumerate(feature_columns, 1):
    print(f"  {i:2d}. {feature}")

# Display sample data
print(f"\nSample data:")
print(final_dataset.head())

# Display statistics
print(f"\nDataset statistics:")
print(final_dataset.describe())

# Check for missing values
missing_counts = final_dataset.isnull().sum()
if missing_counts.sum() > 0:
    print(f"\n⚠️ Missing values found:")
    print(missing_counts[missing_counts > 0])
else:
    print(f"\n✓ No missing values found!")

print(f"\n💾 Saving final training dataset...")
final_dataset.to_csv(data_processed / "final_training_dataset.csv", index=False)
final_dataset.to_parquet(data_processed / "final_training_dataset.parquet", index=False)
print(f"✓ Final dataset saved successfully!")

# Memory usage
memory_usage = final_dataset.memory_usage(deep=True).sum() / 1024**2
print(f"📊 Final dataset memory usage: {memory_usage:.1f} MB")

## 6. Feature Importance Analysis & Visualization

Phân tích tầm quan trọng của các features và visualize relationships:

In [ ]:
print("FEATURE IMPORTANCE ANALYSIS")
print("="*35)

# Prepare data for analysis
analysis_data = final_dataset.copy()
feature_cols = [col for col in analysis_data.columns if col not in ['user_id', 'product_id', 'reordered']]
X = analysis_data[feature_cols]
y = analysis_data['reordered']

print(f"Features for analysis: {len(feature_cols)}")
print(f"Samples: {len(X):,}")

# 1. Random Forest Feature Importance
print("\n🌲 Computing Random Forest feature importance...")
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# Use a sample for faster computation
sample_size = min(50000, len(X))
sample_idx = np.random.choice(len(X), sample_size, replace=False)
X_sample = X.iloc[sample_idx]
y_sample = y.iloc[sample_idx]

rf.fit(X_sample, y_sample)
rf_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 15 Random Forest Features:")
print(rf_importance.head(15))

# 2. Correlation with target
print("\n📊 Computing feature correlations with target...")
correlations = []
for col in feature_cols:
    corr = np.corrcoef(X[col], y)[0, 1]
    correlations.append({'feature': col, 'correlation': abs(corr)})

correlation_df = pd.DataFrame(correlations).sort_values('correlation', ascending=False)
print("Top 15 Correlated Features:")
print(correlation_df.head(15))

# 3. Feature distributions by target
print("\n📈 Creating feature importance visualization...")

# Plot top features
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Random Forest Importance
top_rf_features = rf_importance.head(15)
axes[0, 0].barh(range(len(top_rf_features)), top_rf_features['importance'])
axes[0, 0].set_yticks(range(len(top_rf_features)))
axes[0, 0].set_yticklabels(top_rf_features['feature'])
axes[0, 0].set_title('Top 15 Random Forest Feature Importance')
axes[0, 0].set_xlabel('Importance')

# Correlation with target
top_corr_features = correlation_df.head(15)
axes[0, 1].barh(range(len(top_corr_features)), top_corr_features['correlation'])
axes[0, 1].set_yticks(range(len(top_corr_features)))
axes[0, 1].set_yticklabels(top_corr_features['feature'])
axes[0, 1].set_title('Top 15 Feature Correlations with Target')
axes[0, 1].set_xlabel('Absolute Correlation')

# Distribution of top feature by target
top_feature = rf_importance.iloc[0]['feature']
reorder_yes = analysis_data[analysis_data['reordered'] == 1][top_feature]
reorder_no = analysis_data[analysis_data['reordered'] == 0][top_feature]

axes[1, 0].hist(reorder_no, bins=50, alpha=0.7, label='Not Reordered', density=True)
axes[1, 0].hist(reorder_yes, bins=50, alpha=0.7, label='Reordered', density=True)
axes[1, 0].set_title(f'Distribution of {top_feature}')
axes[1, 0].set_xlabel(top_feature)
axes[1, 0].set_ylabel('Density')
axes[1, 0].legend()

# Feature correlation heatmap (top features only)
top_features_for_corr = rf_importance.head(10)['feature'].tolist()
corr_matrix = analysis_data[top_features_for_corr].corr()
im = axes[1, 1].imshow(corr_matrix, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
axes[1, 1].set_xticks(range(len(top_features_for_corr)))
axes[1, 1].set_yticks(range(len(top_features_for_corr)))
axes[1, 1].set_xticklabels(top_features_for_corr, rotation=45, ha='right')
axes[1, 1].set_yticklabels(top_features_for_corr)
axes[1, 1].set_title('Feature Correlation Matrix (Top 10)')
plt.colorbar(im, ax=axes[1, 1])

plt.tight_layout()
plt.show()

# 4. Save feature importance results
print("\n💾 Saving feature importance analysis...")
rf_importance.to_csv(data_processed / "feature_importance_rf.csv", index=False)
correlation_df.to_csv(data_processed / "feature_correlation.csv", index=False)

# Feature summary
feature_summary = pd.DataFrame({
    'feature': feature_cols,
    'rf_importance': rf_importance.set_index('feature').loc[feature_cols, 'importance'].values,
    'correlation': correlation_df.set_index('feature').loc[feature_cols, 'correlation'].values,
})
feature_summary['combined_score'] = (feature_summary['rf_importance'] * 0.6 + 
                                   feature_summary['correlation'] * 0.4)
feature_summary = feature_summary.sort_values('combined_score', ascending=False)

print("\nTop 20 Features (Combined Score):")
print(feature_summary.head(20))

feature_summary.to_csv(data_processed / "feature_summary.csv", index=False)
print("✓ Feature analysis saved successfully!")

print(f"\n🎯 FEATURE ENGINEERING COMPLETE!")
print(f"📁 All processed data saved to: {data_processed}")
print(f"📊 Ready for model training!")

# Summary statistics
print(f"\n📈 FINAL SUMMARY:")
print(f"  👥 Users: {final_dataset['user_id'].nunique():,}")
print(f"  🛒 Products: {final_dataset['product_id'].nunique():,}")
print(f"  📝 Training samples: {final_dataset.shape[0]:,}")
print(f"  🔢 Features: {len(feature_cols)}")
print(f"  🎯 Reorder rate: {final_dataset['reordered'].mean():.1%}")
print(f"  💾 Dataset size: {final_dataset.memory_usage(deep=True).sum() / 1024**2:.1f} MB")